Reconocedor de lenguaje de señas Argentino entrenado solo con el dataset argentino.

In [3]:
import tensorflow as tf
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt

Cargamos el modelo de manera que sea entrenable. No incluímos su última capa para poder establecer nuestras propias clases.

In [4]:
# Cargamos InceptionV3
base_model = InceptionV3(weights = 'imagenet',       # Pre-entrenado con ImageNet
                         include_top = False,        # Sin incluir su capa de clasificacion con 1000 clases para poder hacer fine-tuning 
                         input_shape = (299, 299, 3) # Necesario cuando no incluimos la ultima capa
                        )

# Inicialmente congelamos todas las capas, despues descongelaremos las que nos interesen
base_model.trainable = False

2025-06-30 15:58:08.671873: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Importamos los datasets y hacemos un split.

In [22]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

# Extraemos los nombres de todas las imagenes que vamos a utilizar
image_dir = 'lsa16_segmented/'
filenames = [f for f in os.listdir(image_dir)]

# Y de cada una extraemos su clase, que viene dada por el primer numero del nombre
labels = [int(f.split('_')[0]) for f in filenames]

# Y creamos un dataframe que asocia a cada nombre de archivo su clase.
df = pd.DataFrame({'filename': filenames, 'class': labels})

       filename  class
0     5_5_2.png      5
1     2_4_5.png      2
2     9_3_2.png      9
3    15_9_4.png     15
4    10_5_3.png     10
..          ...    ...
795   7_6_1.png      7
796  12_9_2.png     12
797  6_10_3.png      6
798   2_6_4.png      2
799   4_1_3.png      4

[800 rows x 2 columns]


Preprocesamos las imagenes para adecuarlas al formato de ImageNet

In [9]:
# Separamos al dataset en train y validacion.
train_df, val_df = train_test_split(df,
                                    test_size=0.2,
                                    stratify=df['class'],   # Hace que se mantengan las proporciones de las clases luego del split
                                    random_state=42)

Creamos un *pipeline* de datos de *TensorFlow*,

In [20]:
# Definimos una funcion que dado un filename devuelve su imagen y su clase o label
def load_and_preprocess(image_path, label):

    # Leemos el archivo y lo decodificamos en RGB
    img = tf.io.read_file(image_dir + image_path)
    img = tf.image.decode_jpeg(img, channels=3) 
    
    # Lo preprocesamos para InceptionV3
    img = tf.image.resize(img, [299, 299])
    img = preprocess_input(img)  # Obs. que preprocess_input es una funcion de inception_v3 en particular
    
    return img, label

# Usamos un batch_size de TensorFlow estandar
batch_size = 32

# 1. Cargamos el dataframe
ds = tf.data.Dataset.from_tensor_slices((train_df['filename'].values, train_df['class'].values))

# 2. Le mappeamos el preprocesamiento a cada entrada, paralelizando
ds = ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# 3. Mezclamos para randomizar el orden de las muestras
ds = ds.shuffle(buffer_size=len(train_df))

# 4. Usamos el batch_size estandar
ds = ds.batch(batch_size)

# 5. Permitimos el prefetching del proximo batch
train_ds = ds.prefetch(tf.data.AUTOTUNE)                                                       

# Y repetimos lo mismo para el conjunto de validacion
val_ds = tf.data.Dataset.from_tensor_slices((val_df['filename'].values, val_df['class'].values))\
           .map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE) \
           .batch(batch_size) \
           .prefetch(tf.data.AUTOTUNE)

In [ ]:
# Definimos la ultima capa para que prediga acorde a nuestras clases
num_classes = train_df['class'].nunique()
print(f"Número de clases: {num_classes}")

x = base_model.output  # x es la salida del modelo hasta ahora
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Regularización para evitar overfitting

# Y creamos una nueva capa de salida que tome como input a la anterior y clasifique en num_classes clases
predictions = Dense(num_classes, activation='softmax')(x)  

model = Model(inputs=base_model.input, outputs=predictions)

Configuramos el modelo usando momento adaptativo y *sparse categorial cross-entropy* pues es adecuada para clasificacion multiclase con enteros.

In [24]:
base_model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',  # Para etiquetas enteras
    metrics=['accuracy']
)

In [ ]:
img_path = 'asl_dataset/a/hand1_a_bot_seg_5_cropped.jpeg'  
img = image.load_img(img_path, target_size=(299, 299)) # La carga en img y le hace resize a 299x299
img_array = image.img_to_array(img)                    # La convierte a array de NumPy con dimensiones (299, 299, 3)
img_array = np.expand_dims(img_array, axis=0)          # Agrega una dimension mas al array, haciendolo (1, 299, 299, 3) para batching
img_array = preprocess_input(img_array)                # Matchea la representacion de la imagen a como la espera ImageNet (ej. mappea 0-255 a -1,1, cambia de RGB a BGR)

# Predict
predictions = model.predict(img_array)
decoded_predictions = decode_predictions(predictions, top=5)[0]

# Display results
plt.imshow(img)
plt.axis('off')
plt.show()

print("Top 5 Predictions:")
for i, (_, label, prob) in enumerate(decoded_predictions):
    print(f"{i + 1}: {label} ({prob * 100:.2f}%)")